### [SEP]付きで学習できるかどうか試すnotebook

* とりあえずduplicatedはそのまま
* text_picked==0はtextを流用する

【メモ】
* dataset_recover.pyだと\tが消える
    * ピックアップの時点で[SEP]入れるべきでは？


In [29]:
import pandas as pd
import numpy as np

from bert_utils import *
from config import *

In [96]:
train_df = pd.read_feather("input/[DEBUG]dataset_with_original_text/train_with_original_text.feather")
train_duplicated_index = pd.read_json("input/[DEBUG]dataset_with_original_text/train_duplicated_index.json", typ="series")

In [31]:
corpus_df_list = []
for source in ["newsplus", "news4vip", "livejupiter"]:
    corpus_df = pd.DataFrame(columns=["id", "source", "text"])
    tsv_line_list = []
    with open(input_root+f"corpus/{source}.tsv", encoding="utf-8") as f:
        for i, l in tqdm(enumerate(f)):
            tsv_line_list.append(l)
    corpus_df["text"] = tsv_line_list
    corpus_df["source"] = source
    corpus_df = corpus_df.reset_index(drop=False)
    corpus_df["id"] = corpus_df["source"] + "_" + corpus_df["index"].astype(str)
    corpus_df = corpus_df.drop(["index"], axis=1)
    corpus_df_list.append(corpus_df)

corpus_df = pd.concat(corpus_df_list)
corpus_df = corpus_df.reset_index(drop=True)

217296it [00:00, 992478.04it/s] 
1983626it [00:01, 1254455.35it/s]
5948218it [00:04, 1369772.10it/s]


In [35]:
train_df

,id,source,text,label,original_text
0,80074aa43,news4vip,まともに相手されてない人との関係なんて\nそんな大事にするものか？,0,まともにLINEの返信してくれないんだけど諦めなきゃダメだろ __BR__ フラれたらギクシ...
1,6378fea6b,livejupiter,最近はアヘアヘQSマンやない？ ｲｲ!(・∀・)+1-0(・Ａ・)ｲｸﾅｲ!,0,柳なんでこんないい時と悪い時の波があるんや\t最近はアヘアヘQSマンやない？ ｲｲ!(・∀・...
2,c535f5613,livejupiter,日本人として生まれても無能な低学歴って分かったら日本人の権利剥奪して追放すべきやろ\n甘えるな,1,日本人として生まれても無能な低学歴って分かったら日本人の権利剥奪して追放すべきやろ __BR...
3,e76638295,livejupiter,よくよく思えば川上は配布にしたらとんでもなく有能だよな\nガチャから引いたら圧倒的歓喜レベルやで,0,よくよく思えば川上は配布にしたらとんでもなく有能だよな __BR__ ガチャから引いたら圧倒...
4,51e4036bf,newsplus,押井は原作レイプの専門家だから\n原作マンガの真意を誤解させることに関してはプロだが\nそれ...,0,押井は原作レイプの専門家だから __BR__ 原作マンガの真意を誤解させることに関してはプロ...
...,...,...,...,...,...
5251,42b5f86b0,news4vip,車じゃなくてもよくない？\nケーブル網を張り巡らせてリフトみたいなのを付けるとか,0,車じゃなくてもよくない？ __BR__ ケーブル網を張り巡らせてリフトみたいなのを付けるとか...
5252,402ce15d9,livejupiter,左やぞ？\n出すなら下水流,0,今からでも野間か松山を出せばいいのに… __BR__ 最近のエルは正直いって期待薄\t左やぞ...
5253,0739a9fcb,newsplus,日本によって、けんけんガクガクの議論を持たらされた韓国は被害者\n\n日本人がしっかり考えな...,0,日本によって、けんけんガクガクの議論を持たらされた韓国は被害者 __BR__ __BR__ ...
5254,d496c7dc8,news4vip,ゴムボート買って、沖まで漕いで行ったら？\n魚の血を塗っておけばサメが食べてくれるよ,0,安楽死を合法にしたら若者殺到するんだろうなあ\tゴムボート買って、沖まで漕いで行ったら？ _...


In [39]:
t = [x for x in corpus_df["text"].values if "まともに相手されてない人" in x][0]

In [41]:
tt = t

In [55]:
ttt = tt.replace("\t", "[SEP]").replace(" __BR__ ", "。").replace("\n", "。")
ttt = "[CLS]"+ttt
ttt

'[CLS]まともにLINEの返信してくれないんだけど諦めなきゃダメだろ。フラれたらギクシャクするだろ[SEP]まともに相手されてない人との関係なんて。そんな大事にするものか？[SEP]そこそこ仲良いんだよグループでは。'

In [58]:
tttt = clean_text(ttt)

'[CLS]まともにLINEの返信してくれないんだけど諦めなきゃダメだろ。フラれたらギクシャクするだろ[SEP]まともに相手されてない人との関係なんて。そんな大事にするものか？[SEP]そこそこ仲良いんだよグループでは。'

In [61]:
ttttt = juman_parse(tttt)

### 非juman系

In [66]:
tokenizer = define_tokenizer("cl-tohoku/bert-base-japanese")
tokenizer

PreTrainedTokenizer(name_or_path='cl-tohoku/bert-base-japanese', vocab_size=32000, model_max_len=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [67]:
tokenizer.tokenize(tttt)

['[CLS]',
 'まとも',
 'に',
 'LINE',
 'の',
 '返',
 '##信',
 'し',
 'て',
 'くれ',
 'ない',
 'ん',
 'だ',
 'けど',
 '諦め',
 'なき',
 '##ゃ',
 'ダメ',
 'だろ',
 'フラ',
 'れ',
 'たら',
 'ギ',
 '##クシ',
 '##ャ',
 '##ク',
 'する',
 'だろ',
 '[SEP]',
 'まとも',
 'に',
 '相手',
 'さ',
 'れ',
 'て',
 'な',
 'い',
 '##人',
 'と',
 'の',
 '関係',
 'なんて',
 'そんな',
 '大事',
 'に',
 'する',
 'もの',
 'か',
 '?',
 '[SEP]',
 'そこ',
 '##そ',
 '##こ',
 '仲良',
 '##い',
 'ん',
 'だ',
 'よ',
 'グループ',
 'で',
 'は']

### juman系

In [88]:
t

'まともにLINEの返信してくれないんだけど諦めなきゃダメだろ __BR__ フラれたらギクシャクするだろ\tまともに相手されてない人との関係なんて __BR__ そんな大事にするものか？\tそこそこ仲良いんだよグループでは\n'

* clean_textは\tを残す設計になってる

In [89]:
tt = clean_text(t)
tt

'まともにLINEの返信してくれないんだけど諦めなきゃダメだろ__BR__フラれたらギクシャクするだろ\tまともに相手されてない人との関係なんて__BR__そんな大事にするものか?\tそこそこ仲良いんだよグループでは'

In [91]:
ttt = tt.replace("__BR__", "").replace("\t", "SEP")
ttt = "CLS" + ttt
ttt

'CLSまともにLINEの返信してくれないんだけど諦めなきゃダメだろフラれたらギクシャクするだろSEPまともに相手されてない人との関係なんてそんな大事にするものか?SEPそこそこ仲良いんだよグループでは'

In [92]:
tttt = juman_parse(ttt)
tttt

'CLS まともに LINE の 返信 して くれ ない んだ けど 諦め なきゃ ダメだろ フラれたら ギクシャク する だろ SEP まともに 相手 さ れて ない 人 と の 関係 なんて そんな 大事に する もの か ? SEP そこそこ 仲 良い んだ よ グループ で は'

In [93]:
ttttt =tttt.replace("SEP", "[SEP]").replace("CLS", "[CLS]")
ttttt

'[CLS] まともに LINE の 返信 して くれ ない んだ けど 諦め なきゃ ダメだろ フラれたら ギクシャク する だろ [SEP] まともに 相手 さ れて ない 人 と の 関係 なんて そんな 大事に する もの か ? [SEP] そこそこ 仲 良い んだ よ グループ で は'

In [68]:
tokenizer = define_tokenizer("nlp-waseda/roberta-large-japanese-seq512")
tokenizer

PreTrainedTokenizerFast(name_or_path='nlp-waseda/roberta-large-japanese-seq512', vocab_size=32000, model_max_len=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '[CLS]', 'eos_token': '[SEP]', 'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': AddedToken("[MASK]", rstrip=False, lstrip=True, single_word=False, normalized=False)})

* jumanのあとにspecial-tokenを入れないとカッコをバチ切りされる
    * clean_text -> juman -> insert-special-token -> tokenize

In [95]:
tokenizer.convert_tokens_to_ids(tokenizer.tokenize(ttttt))

[2,
 23108,
 333,
 13173,
 261,
 22364,
 276,
 6496,
 304,
 4736,
 4659,
 11795,
 29159,
 15364,
 1015,
 1830,
 4911,
 5172,
 364,
 3661,
 23862,
 481,
 278,
 524,
 1830,
 275,
 3,
 23108,
 333,
 1427,
 274,
 293,
 304,
 306,
 267,
 261,
 531,
 10667,
 3299,
 26345,
 278,
 323,
 343,
 943,
 275,
 3,
 903,
 3537,
 1094,
 3951,
 2385,
 4736,
 1303,
 712,
 269,
 265]